In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import average_precision_score,precision_recall_curve,f1_score,accuracy_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.externals import joblib


data_train = pd.read_csv("train.csv")

###Replace Missing Values with median for catagorical Columns


data_train.B.fillna(28.17,inplace = True)
data_train.N.fillna(152,inplace = True)

#Treating Missing Value as a neew_catagory for catagorial columns

data_train.fillna('NN',inplace = True)

### saving the target varaible

target = data_train.P

### Columns D is same as E so we can drop any of them. Here dropping D and Keeping E.
### Column J is irrevent for this classification problem. So drop J.
data_train.drop(['D','J','P'],axis =1,inplace =True)


data_train_int = data_train.select_dtypes(include=['float64','int64']).copy()

data_train_cat = data_train.select_dtypes(include=['object']).copy()

train_colums = data_train_cat.columns
label_dict ={}

for each in train_colums:
    la = LabelEncoder().fit(data_train_cat[each])
    data_train_cat[each] = la.transform(data_train_cat[each])
   
    label_dict[each] = la
data_train_cat = data_train_cat.as_matrix()


one_he = OneHotEncoder()


data_train_X =one_he.fit_transform(data_train_cat)
data_train_X =data_train_X.toarray()
data_train_int = data_train_int.as_matrix()
X_train = np.concatenate((data_train_int[:,1::],data_train_X ), axis=1)
Y_train = target.as_matrix()
#X = np.concatenate((data_train_int[:,1::],data_train_X ), axis=1)
#Y = target.as_matrix()

#from sklearn.model_selection import train_test_split
#seed = 10
#test_size = 0.3
#X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=seed)
#X_train = X_train_al[:,1:data_set_one_hot.shape[1]]
#X_test = X_test_al[:,1:data_set_one_hot.shape[1]]



xgb_model = XGBClassifier(booster='gbtree',max_depth=4,n_estimators=1000,learning_rate=0.055)
xgb_model.fit(X_train, Y_train)

#model = XGBClassifier(booster='gbtree',max_depth=4,n_estimators=900,learning_rate=0.055)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)
#predictions = [round(value) for value in y_pred]
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))


### Saving the Model for future usage
filename = 'MMT_classification_final.pkl'
joblib.dump(xgb_model, filename)






data_test = pd.read_csv("test.csv")

ids_list = data_test.id

data_test.drop(['D','J','id'],axis =1,inplace =True)
data_test.B.fillna(28.17,inplace = True)
data_test.N.fillna(152,inplace = True)


data_test.fillna('NN',inplace = True)





data_test_int = data_test.select_dtypes(include=['float64','int64']).copy()

data_test_cat = data_test.select_dtypes(include=['object']).copy()

test_colums = data_test_cat.columns

for each in test_colums:
    la = label_dict[each]
    data_test_cat[each] = la.transform(data_test_cat[each])
   
   

data_test_cat = data_test_cat.as_matrix()



data_test_X =one_he.transform(data_test_cat)
data_test_X =data_test_X.toarray()
data_test_int = data_test_int.as_matrix()
X_test = np.concatenate((data_test_int,data_test_X ), axis=1)



# some time later...

# load the model from disk
#loaded_model = joblib.load(filename)
#result = loaded_model.score(X_test, Y_test)
#print(result)


predicted_class = xgb_model.predict(X_test)


result = pd.DataFrame({'id': ids_list,'P': predicted_class},columns =['id','P'])
result.to_csv('submission.csv',index = False)

/home/hbbg/shyam_works/vir_env/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/home/hbbg/shyam_works/vir_env/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/hbbg/shyam_works/vir_env/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behavio

In [2]:
result.dtypes

id    int64
P     int64
dtype: object

In [3]:
aa =  pd.read_csv("sample_submission.csv")

In [4]:
aa.dtypes

id    int64
P     int64
dtype: object